In [3]:
!pip install streamlit pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00


In [16]:
# app.py

import streamlit as st
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import pytz # For timezone handling

# --- Configuration ---
API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "RST38746G38B7RB46GBER" # Our team's API Key
DEFAULT_QUERY_TEXT = "President of US" # Reliable query
DEFAULT_RESULT_SIZE = 20 # Keep it small for reliability

# --- Function to fetch and process data ---
@st.cache_data # Cache data to avoid re-running API call on every interaction
def fetch_and_process_data(query_text, result_size):
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_KEY
    }
    payload = {
      "query_text": query_text,
      "result_size": result_size,
      "include_highlights": True,
      "ai_answer": "basic"
    }

    try:
        response = requests.post(API_URL, headers=headers, data=json.dumps(payload), timeout=30) # Add timeout
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        json_response = response.json()

        if 'results' in json_response:
            df = pd.json_normalize(json_response['results'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
            df.dropna(subset=['timestamp'], inplace=True)
            df = df.sort_values(by='timestamp', ascending=True).reset_index(drop=True)

            # Get overall AI brief, handling potential API error
            overall_ai_brief = json_response.get('ai_answer', {}).get('text', "AI summary not available or encountered an API error.")
            if "Unable to generate AI answer due to API error" in overall_ai_brief:
                overall_ai_brief = "AI summary encountered an error from the API. We'll focus on the raw article data."

            return df, overall_ai_brief
        else:
            st.error(f"API Response did not contain 'results'. Response: {json_response}")
            return pd.DataFrame(), "No articles retrieved. API response structure unexpected."
    except requests.exceptions.Timeout:
        st.error("API request timed out. Displaying cached data if available, or try a smaller result size.")
        return pd.DataFrame(), "API request timed out. Data not retrieved."
    except requests.exceptions.RequestException as e:
        st.error(f"Error making API request: {e}. Displaying cached data if available.")
        return pd.DataFrame(), f"Error retrieving data: {e}"
    except json.JSONDecodeError:
        st.error("Could not decode JSON from API response.")
        return pd.DataFrame(), "Error: API response was not valid JSON."

# --- Function to filter DataFrame by time ---
def filter_df_by_time(dataframe, num_units, unit_type):
    london_tz = pytz.timezone('Europe/London')
    current_date = datetime.now(london_tz)

    if unit_type == 'days':
        start_date = current_date - timedelta(days=num_units)
    elif unit_type == 'months':
        start_date = current_date - timedelta(days=num_units * 30) # Approximate
    elif unit_type == 'years':
        start_date = current_date - timedelta(days=num_units * 365) # Approximate
    else:
        st.warning("Invalid unit_type. Please choose 'days', 'months', or 'years'.")
        return dataframe

    # Ensure DataFrame timestamps are timezone-aware and converted to London timezone
    if dataframe['timestamp'].dt.tz is None:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_localize('UTC').dt.tz_convert(london_tz)
    else:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_convert(london_tz)

    filtered_df = dataframe[dataframe['timestamp'] >= start_date].copy()
    return filtered_df.sort_values(by='timestamp', ascending=True)

# --- Streamlit Application Layout ---
st.set_page_config(layout="wide") # Use wide layout for better display

st.title("Amplify Chronos: Public Discourse Timeline")
st.markdown("Track the chronological development of news topics using the Amplyfi API.")

# Sidebar for controls
with st.sidebar:
    st.header("Timeline Controls")

    # Dynamic query text input
    user_query = st.text_input("Enter Topic Query:", value=DEFAULT_QUERY_TEXT)
    # Option to refresh data
    if st.button("Refresh Data"):
        st.experimental_rerun() # Forces a rerun of the script to fetch new data

    st.subheader("Filter Timeline by Time")
    num_units = st.number_input(
        "Go back by (number):",
        min_value=1,
        value=7, # Default to 7 days
        step=1
    )
    unit_type = st.selectbox(
        "Time Unit:",
        ('days', 'months', 'years'),
        index=0 # Default to 'days'
    )

# --- Fetch and process data ---
full_df, overall_ai_brief = fetch_and_process_data(user_query, DEFAULT_RESULT_SIZE)

# Display overall AI Brief (if available)
st.subheader(f"Overall Brief for '{user_query}':")
st.info(overall_ai_brief)

if not full_df.empty:
    st.subheader(f"Chronological Timeline for '{user_query}'")

    # Apply filtering
    filtered_df = filter_df_by_time(full_df, num_units, unit_type)

    if not filtered_df.empty:
        # Display each article
        for index, row in filtered_df.iterrows():
            st.write(f"**[{row['timestamp'].strftime('%Y-%m-%d %H:%M %Z')}]**")
            st.write(f"### {row['title']}")
            st.write(f"*{row['summary']}*")
            st.markdown(f"[Read more]({row['url']})")
            if row['highlights']:
                with st.expander("Show Highlights"):
                    for highlight in row['highlights']:
                        st.markdown(f"- {highlight}")
            st.markdown("---")
    else:
        st.write(f"No articles found for '{user_query}' in the last {num_units} {unit_type}.")
else:
    st.warning("No data available to display the timeline. Please check the API status or query.")

2025-06-10 13:11:57.899 No runtime found, using MemoryCacheStorageManager
2025-06-10 13:11:57.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 13:11:57.911 Thread 'MainThread':

In [17]:
# --- Hardcoded Fallback Data (from your previous successful output) ---
# This approach uses a multi-line string and json.loads() for robustness
def get_hardcoded_data_for_console():
    # The raw JSON as a multi-line string (triple quotes are key here)
    raw_json_string = """
{'results': [{'id': 'DL-e4b57b2b5b5b0d8d590baa5548e5b1b4',
   'title': "When Billionaires Collide: Trump vs Musk's Social Media Showdown",
   'url': 'https://www.devdiscourse.com/article/politics/3449428-when-billionaires-collide-trump-vs-musks-social-media-showdown',
   'summary': "Elon Musk accuses Trump of betrayal over unfulfilled promises and suggesting impeachment. Trump's retorts were equally fierce, hinting at the severance of financial ties.",
   'score': 137.53534,
   'timestamp': '2025-06-06T12:21:56',
   'highlights': []},
  {'id': 'DL-29dee045bc94525e2ffde561e23d9a0e',
   'title': 'USA“Trump is in the Epstein files”: Elon Musk and Donald Trump flirt on social media',
   'url': 'https://www.tageblatt.lu/headlines/trump-ist-in-den-epstein-akten-elon-musk-und-donald-trump-zoffen-sich-in-sozialen-medien/',
   'summary': "Elon Musk and Donald Trump are in public – and it's personal. The months-long alliance between Musk and Trump ends in a public mudslide. Trump threatened to remove Musk's company from government contracts.",
   'score': 110.60926,
   'timestamp': '2025-06-06T04:53:17+00:00',
   'highlights': ['Trump said Musk had no problem with the law – until he learned that it included cutting billions in subsidies for electric vehicles. Musk called it a lie.',
    'Musk: Trump will only be president for three and a half years The 53-year-old owner of the large-scale online platform X suggested to congressmen of the Republican Party to side with him in the vote on the law.',
    '“The easiest way to save billions and billions of dollars in our budget is to cancel Elon’s government subsidies and contracts,” the president wrote on the online platform Truth Social.']},
  {'id': 'DL-32d399c822715c2b58cd184fa438c168',
   'title': "Sen. Warren asks for contingency plans on national security after Trump and Musk's social media fall out",
   'url': 'https://flipboard.com/article/sen-warren-asks-for-contingency-plans-on-national-security-after-trump-and-musk/f-2ef1d09686%2Fcbsnews.com',
   'summary': "Sen. Elizabeth Warren is asking Secretary of State Marco Rubio for information on the Trump administration's contingency plans if billionaire Elon …",
   'score': 103.89935,
   'timestamp': '2025-06-08T03:31:16+00:00',
   'highlights': []},
  {'id': 'DL-c736d5dced3a423ad97a99d3a9065efe',
   'title': 'Trump and Musk battle begins: 8 ways they can finish each other off',
   'url': 'https://www.tgrthaber.com/dunya/trump-ve-musk-savasi-basladi-birbirlerini-bitirebilecekleri-8-yol-3225158',
   'summary': 'Donald Trump and Elon Musk clashed on social media yesterday. There are eight main ways for the aggressive couple to "punish" each other. Musk and Trump\'s "conspiracy" has gained widespread attention in the media.',
   'score': 103.47779,
   'timestamp': '2025-06-06T10:40:46',
   'highlights': ['The war between US President Donald Trump and billionaire businessman Elon Musk is heating up, according to US media reports. There are eight main ways for the constantly aggressive couple to "punish" each other.',
    'US President Donald Trump and billionaire businessman Elon Musk clashed on social media yesterday.',
    'Trump also told his social media platform Truth Social that Musk-owned companies, including SpaceX and the automaker Tesla, could end their deals with the government. This "will be the easiest way to save on the budget," he added.']},
  {'id': 'DL-cb3318bd320f7cbbdeabca5ba04c483b',
   'title': 'Epstein file accusations, potential contract cuts, impeachment: Trump-Musk feud explodes',
   'url': 'https://www.abc57.com/news/epstein-file-accusations-potential-contract-cuts-impeachment-trump-musk-feud-explodes',
   'summary': 'Donald Trump and Elon Musk sparred on their respective social media platforms. Trump confirmed the deterioration of his relationship with Musk. Musk responded on his social media platform X that Trump could not have won the 2024 election without him.',
   'score': 95.47830,
   'timestamp': '2025-06-06T03:22:00+00:00',
   'highlights': ['President Donald Trump and Elon Musk explosively sparred on their respective social media platforms Thursday – with the president floating the idea of cutting the tech billionaire’s various government contracts and Musk going nuclear at one point and',
    'Shortly after, Musk responded on his social media platform X that Trump could not have won the 2024 election without him – a jab that appeared to further irritate the president and significantly intensify the fight, which played out on dueling social',
    '“Without me, Trump would have lost the election, Dems would control the House and the Republicans would be 51-49 in the Senate,” Musk said. He added: “Such ingratitude.”']},
  {'id': 'DL-44785917f97e5c8f4cbd700392c06b09',
   'title': 'Elon Musk squeezes first shot! Trump shares party footage with Epstein',
   'url': 'https://www.tgrthaber.com/dunya-videolari/elon-musk-ilk-kursunu-sikti-trumpin-epstein-ile-parti-yaptigi-goruntuleri-paylasti-3225172',
   'summary': "Elon Musk squeezed his first shot! US President Donald Trump and billionaire businessman Elon Musk clashed on social media yesterday, sharing footage of Trump partying with Epstein. Tensions between the two have been reported in the media, with Musk citing Trump's name in the Jeffrey Epstein case and Trump indicating that the US government could end deals with Trump's companies. Following the debate, Elon Musk shared footage of Donald Trump's party with Jeffrey Epstein in 1992.",
   'score': 88.88725,
   'timestamp': '2025-06-06T11:16:39',
   'highlights': ['Elon Musk squeezed his first shot! US President Donald Trump and billionaire businessman Elon Musk clashed on social media yesterday, sharing footage of Trump partying with Epstein.',
    "Tensions between the two have been reported in the media, with Musk citing Trump's name in the Jeffrey Epstein case and Trump indicating that the US government could end deals with Trump's companies.",
    "Following the debate, Elon Musk shared footage of Donald Trump's party with Jeffrey Epstein in 1992."]},
  {'id': 'DL-b6bfb32f02817bea5a62ae4b521ffb35',
   'title': 'Disappointment and lies: Trump and Musk quarrel on social media',
   'url': 'https://eadaily.com/ru/news/2025/06/05/razocharovanie-i-lozh-tramp-i-mask-possorilis-v-socsetyah',
   'summary': 'Donald Trump is "disappointed" by Ilon Musk because of his criticism of the budget bill. He noted that Mack knew all the internal work on the document and all its advantages. Mask promptly responded to this statement.',
   'score': 80.47531,
   'timestamp': '2025-06-05T17:24:00+00:00',
   'highlights': []},
  {'id': 'DL-f74238144d442af0d0135f002ecfabd0',
   'title': 'Trump and Musk swap accusations on social media',
   'url': 'https://www.rtp.pt/noticias/mundo/trump-e-musk-trocam-acusacoes-nas-redes-sociais_v1660534',
   'summary': "Elon Musk claimed that Donald Trump's name appears on the Epstein files. It's a sexual scandal involving the prostitution and exploitation of minors.",
   'score': 73.79056,
   'timestamp': '2025-06-06T20:41:48',
   'highlights': []},
  {'id': 'DL-eb731bfcade6d2a5ec4aad4f294069e5',
   'title': "Musk's father announces in Moscow that Elon made a mistake due to stress, Trump will quit",
   'url': 'https://observatornews.ro/extern/tatal-lui-musk-anunta-de-la-moscova-ca-elon-a-gresit-din-cauza-stresului-si-ca-trump-va-birui-621931.html',
   'summary': "Errol Musk, Elon Musk's father, defended Donald Trump in a public dispute with his son. Elon reacted wrongly due to stress and that Trump would go to work, he says. Musk and Trump began insulting each other on social media last week, with Musk denouncing the president's expenditure tax and bill.",
   'score': 65.28914,
   'timestamp': None,
   'highlights': ["Elon Musk's father, Errol, in Moscow, where he attends a patriotic forum organised by Aleksandr Dughin, 7 June 2025 - The conflict between Elon Musk, the world's richest man, and US President Donald Trump was triggered by the stress of both, and Elon",
    "made a mistake by publicly challenging Trump, Musk's father, Errol, told Russian media in Moscow, EFE reported.",
    'Musk and Trump began insulting each other on social media last week, with Musk denouncing the president\'s expenditure tax and bill as a "disgusting hoax".']},
  {'id': 'DL-17463cc334ad75cabe76165740a9e398',
   'title': "Donald Trump and Elon Musk's bromance comes to bitter end as public feud erupts on social media",
   'url': 'https://www.manchestereveningnews.co.uk/news/world-news/donald-trump-elon-musks-bromance-31799516',
   'summary': 'Trump threatened to cut Elon Musk’s government contracts. Elon said he would use the US government to hurt his fellow billionaire financially. Mr Trump clapped back on Thursday (June 5) in the Oval Office.',
   'score': 61.96383,
   'timestamp': '2025-06-05T21:14:14+00:00',
   'highlights': ["The spectacular blow-up between the world’s richest man and the American president played out on social media\nDonald Trump and Elon Musk's bromance appears to have come to an end as a feud between the Tesla boss and US President erupted on social media",
    'Mr Trump had largely remained silent as Mr Musk stewed over the last few days on his social media platform X, condemning the president’s signature tax cuts and spending bill.',
    "But Mr Trump clapped back on Thursday (June 5) in the Oval Office, lamented their frayed relationship and said he was 'very disappointed in Musk', while Mr Musk responded on social media in real time."]}],
 'query_details': {'query_text': 'trump musk social media',
  'rerank_method': None,
  'diversity_ratio': 0,
  'result_size': 10,
  'timerange': '7d',
  'include_highlights': True,
  'include_smart_tags': None,
  'ai_answer': 'basic',
  'cluster_results': None},
 'count': 728,
 'ai_answer': {'text': "Donald Trump and Elon Musk have been involved in a public feud on social media, with accusations and threats exchanged. Musk accused Trump of betrayal and suggested impeachment, while Trump hinted at severing financial ties and threatened to remove Musk's company from government contracts. This conflict has gained significant media attention, with various strategies discussed on how they could 'punish' each other. Musk claimed that Trump could not have won the 2024 election without his support. The feud escalated on June 6, 2025, with multiple articles covering the developments. For more details, see [When Billionaires Collide: Trump vs Musk's Social Media Showdown](https://www.devdiscourse.com/article/politics/3449428-when-billionaires-collide-trump-vs-musks-social-media-showdown), [USA “Trump is in the Epstein files”: Elon Musk and Donald Trump flirt on social media](https://www.tageblatt.lu/headlines/trump-ist-in-den-epstein-akten-elon-musk-und-donald-trump-zoffen-sich-in-sozialen-medien/), [Trump and Musk battle begins: 8 ways they can finish each other off](https://www.tgrthaber.com/dunya/trump-ve-musk-savasi-basladi-birbirlerini-bitirebilecekleri-8-yol-3225158), and [Epstein file accusations, potential contract cuts, impeachment: Trump-Musk feud explodes](https://www.abc57.com/news/epstein-file-accusa...Donald Trump and Elon Musk have been involved in a public feud on social media, with accusations and threats exchanged. Musk accused Trump of betrayal and suggested impeachment, while Trump hinted at severing financial ties and threatened to remove Musk%27s company from government contracts. This conflict has gained significant media attention, with various strategies discussed on how they could %27punish%27 each other. Musk claimed that Trump could not have won the 2024 election without his support. The feud escalated on June 6, 2025, with multiple articles covering the developments. For more details, see [When Billionaires Collide: Trump vs Musk%27s Social Media Showdown](https://www.devdiscourse.com/article/politics/3449428-when-billionaires-collide-trump-vs-musks-social-media-showdown), [USA %E2%80%9CTrump is in the Epstein files%E2%80%9D: Elon Musk and Donald Trump flirt on social media](https://www.tageblatt.lu/headlines/trump-ist-in-den-epstein-akten-elon-musk-und-donald-trump-zoffen-sich-in-sozialen-medien/), [Trump and Musk battle begins: 8 ways they can finish each other off](https://www.tgrthaber.com/dunya/trump-ve-musk-savasi-basladi-birbirlerini-bitirebilecekleri-8-yol-3225158), and [Epstein file accusations, potential contract cuts, impeachment: Trump-Musk feud explodes](https://www.abc57.com/news/epstein-file-accusa...Donald Trump and Elon Musk have been involved in a public feud on social media, with accusations and threats exchanged. Musk accused Trump of betrayal and suggested impeachment, while Trump hinted at severing financial ties and threatened to remove Musk%27s company from government contracts. This conflict has gained significant media attention, with various strategies discussed on how they could %27punish%27 each other. Musk claimed that Trump could not have won the 2024 election without his support. The feud escalated on June 6, 2025, with multiple articles covering the developments. For more details, see [When Billionaires Collide: Trump vs Musk%27s Social Media Showdown](https://www.devdiscourse.com/article/politics/3449428-when-bil... ',
 'source_document_ids': ['DL-e4b57b2b5b5b0d8d590baa5548e5b1b4',
  'DL-29dee045bc94525e2ffde561e23d9a0e',
  'DL-32d399c822715c2b58cd184fa438c168',
  'DL-c736d5dced3a423ad97a99d3a9065efe',
  'DL-cb3318bd320f7cbbdeabca5ba04c483b']}}
"""
    try:
        # Use ast.literal_eval for safer parsing of Python literals, then json.dumps and json.loads
        import ast
        # First, safely evaluate the string to a Python dictionary literal
        dict_literal = ast.literal_eval(raw_json_string)
        # Then, convert it to a JSON string and load it properly with json.loads
        hardcoded_json_response = json.loads(json.dumps(dict_literal))
    except (SyntaxError, ValueError, TypeError) as e:
        print(f"Error parsing hardcoded JSON string: {e}")
        # Fallback to a minimal safe structure if parsing fails
        hardcoded_json_response = {'results': [], 'query_details': {}, 'count': 0, 'ai_answer': {'text': "Error loading static data. Please check raw_json_string.", 'source_document_ids': []}}

    df_hardcoded = pd.json_normalize(hardcoded_json_response['results'])
    overall_ai_brief_hardcoded = hardcoded_json_response.get('ai_answer', {}).get('text', "AI summary not available.")
    if "Unable to generate AI answer due to API error" in overall_ai_brief_hardcoded:
        overall_ai_brief_hardcoded = "AI summary encountered an error from the API. We'll focus on the raw article data."

    return df_hardcoded, overall_ai_brief_hardcoded


# --- Function to filter DataFrame by time (same as before) ---
def filter_df_by_time(dataframe, num_units, unit_type):
    london_tz = pytz.timezone('Europe/London')
    current_date = datetime.now(london_tz)

    if unit_type == 'days':
        start_date = current_date - timedelta(days=num_units)
    elif unit_type == 'months':
        start_date = current_date - timedelta(days=num_units * 30) # Approximate
    elif unit_type == 'years':
        start_date = current_date - timedelta(days=num_units * 365) # Approximate
    else:
        print("Invalid unit_type. Please choose 'days', 'months', or 'years'.")
        return dataframe

    # Ensure DataFrame timestamps are timezone-aware and converted to London timezone
    if dataframe['timestamp'].dt.tz is None:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_localize('UTC').dt.tz_convert(london_tz)
    else:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_convert(london_tz)

    filtered_df = dataframe[dataframe['timestamp'] >= start_date].copy()
    return filtered_df.sort_values(by='timestamp', ascending=True)

# --- Main execution block for console output ---
if __name__ == "__main__":
    print("--- Amplify Chronos: Public Discourse Timeline (Console Version) ---")

    # Fetch data (will try live API with DEFAULT_QUERY_TEXT, fallback to hardcoded)
    # Use the get_hardcoded_data_for_console function here
    full_df, overall_ai_brief = get_hardcoded_data_for_console()

    if full_df.empty:
        print("\nCould not retrieve any articles. Please check the API status or your query.")
    else:
        # Display overall AI Brief
        print(f"\nOverall Brief for '{DEFAULT_QUERY_TEXT}':")
        print(overall_ai_brief)
        print("-" * 50)

        # Get user input for filtering
        print("\nEnter filter parameters:")
        while True:
            try:
                num_units = int(input("Go back by (number): "))
                if num_units <= 0:
                    raise ValueError
                break
            except ValueError:
                print("Invalid input. Please enter a positive whole number.")

        while True:
            unit_type = input("Time Unit (days, months, years): ").lower()
            if unit_type in ['days', 'months', 'years']:
                break
            else:
                print("Invalid unit type. Please choose 'days', 'months', or 'years'.")

        # Apply filtering
        filtered_df = filter_df_by_time(full_df, num_units, unit_type)

        print(f"\n--- Chronological Timeline for '{DEFAULT_QUERY_TEXT}' (Last {num_units} {unit_type}) ---")
        print(f"Total articles in this filtered period: {len(filtered_df)}")
        print(f"Filtering from current time: {datetime.now(pytz.timezone('Europe/London')).strftime('%Y-%m-%d %H:%M:%S %Z%z')}")
        print("-" * 50)

        if not filtered_df.empty:
            for index, row in filtered_df.iterrows():
                formatted_timestamp = row['timestamp'].strftime('%Y-%m-%d %H:%M %Z')
                print(f"[{formatted_timestamp}]")
                print(f"Title: {row['title']}")
                print(f"Summary: {row['summary']}")
                print(f"URL: {row['url']}")
                if row['highlights']:
                    print("  Highlights:")
                    for highlight in row['highlights']:
                        print(f"  - {highlight}")
                print("-" * 50)
        else:
            print(f"No articles found for '{DEFAULT_QUERY_TEXT}' in the last {num_units} {unit_type}.")

--- Amplify Chronos: Public Discourse Timeline (Console Version) ---
Error parsing hardcoded JSON string: unterminated string literal (detected at line 28) (<unknown>, line 28)

Could not retrieve any articles. Please check the API status or your query.


In [22]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import pytz # For timezone handling

# --- Configuration ---
# API_URL and API_KEY are now effectively symbolic as live calls are failing
API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "RST387746G38B7RB46GBER" # Keep for completeness, but won't be used for live calls
DEFAULT_QUERY_TEXT = "President of US (Using static data)" # Updated to reflect static data
DEFAULT_RESULT_SIZE = 20 # Not directly used for static data, but kept for context

# --- Function to fetch and process data (now only calls fallback) ---
# We are effectively forcing the use of hardcoded data due to 403 Forbidden.
def fetch_and_process_data_for_console(query_text, result_size):
    print("Live API access is currently forbidden (403 error). Falling back to static data for demo purposes.")
    # Always get hardcoded data
    df, brief = get_hardcoded_data_for_console()
    return df, brief


# --- Hardcoded Fallback Data (MINIMAL, GUARANTEED-VALID JSON) ---
# This version is simplified to ensure JSON validity and quick startup.
def get_hardcoded_data_for_console():
    raw_json_string = """
    {
      "results": [
        {
          "id": "DL-simplified-1",
          "title": "Musk Says Trump Won't Be President Without His Help",
          "url": "https://example.com/musk-trump-help",
          "summary": "Elon Musk claimed Donald Trump would not achieve presidency without his support, sparking public debate.",
          "score": 250.0,
          "timestamp": "2025-06-05T14:10:00+00:00",
          "highlights": ["Musk: Trump will not become president without my help."]
        },
        {
          "id": "DL-simplified-2",
          "title": "Trump Advised Greta Thunberg on Anger Management",
          "url": "https://example.com/trump-greta-anger",
          "summary": "President Trump publicly suggested Greta Thunberg take anger management courses, drawing mixed reactions.",
          "score": 280.0,
          "timestamp": "2025-06-10T03:16:00+00:00",
          "highlights": ["Trump advised Greta Thunberg to take anger management courses."]
        },
        {
          "id": "DL-simplified-3",
          "title": "Biden's VP Hopes Musk Returns to 'Our Ranks'",
          "url": "https://example.com/vp-musk-ranks",
          "summary": "Vice President Vance expressed hope that Elon Musk would re-engage with their political circle.",
          "score": 190.0,
          "timestamp": "2025-06-07T15:30:00+00:00",
          "highlights": ["Vice President Vance hopes Musk will return to our ranks."]
        }
      ],
      "query_details": {
        "query_text": "simplified static data",
        "result_size": 3,
        "timerange": "7d"
      },
      "count": 3,
      "ai_answer": {
        "text": "This is a static, simplified summary due to live API access issues. It demonstrates interactions between key public figures.",
        "source_document_ids": ["DL-simplified-1", "DL-simplified-2", "DL-simplified-3"]
      }
    }
    """
    try:
        hardcoded_json_response = json.loads(raw_json_string)
    except json.JSONDecodeError as e:
        print(f"CRITICAL ERROR: Failed to parse internal hardcoded JSON string: {e}")
        return pd.DataFrame(), "CRITICAL ERROR: Failed to load static data. Cannot proceed."

    df_hardcoded = pd.json_normalize(hardcoded_json_response['results'])

    # --- IMPORTANT: Convert timestamp to datetime AND handle NaT here ---
    df_hardcoded['timestamp'] = pd.to_datetime(df_hardcoded['timestamp'], errors='coerce')
    df_hardcoded.dropna(subset=['timestamp'], inplace=True)
    # --- End of IMPORTANT timestamp handling ---

    overall_ai_brief_hardcoded = hardcoded_json_response.get('ai_answer', {}).get('text', "AI summary not available in static data.")

    return df_hardcoded, overall_ai_brief_hardcoded


# --- Function to filter DataFrame by time (same as before, requires pytz) ---
def filter_df_by_time(dataframe, num_units, unit_type):
    london_tz = pytz.timezone('Europe/London')
    current_date = datetime.now(london_tz)

    if unit_type == 'days':
        start_date = current_date - timedelta(days=num_units)
    elif unit_type == 'months':
        start_date = current_date - timedelta(days=num_units * 30) # Approximate
    elif unit_type == 'years':
        start_date = current_date - timedelta(days=num_units * 365) # Approximate
    else:
        print("Invalid unit_type. Please choose 'days', 'months', or 'years'.")
        return dataframe

    # We removed the 'timestamp' check here because it's now handled in data fetching functions.
    # Ensure DataFrame timestamps are timezone-aware and converted to London timezone
    # All timestamps should already be datetime objects and NaT-free due to get_hardcoded_data_for_console
    if dataframe['timestamp'].dt.tz is None:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_localize('UTC').dt.tz_convert(london_tz)
    else:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_convert(london_tz)


    filtered_df = dataframe[dataframe['timestamp'] >= start_date].copy()
    return filtered_df.sort_values(by='timestamp', ascending=True)

# --- Main execution block for console output ---
if __name__ == "__main__":
    print("--- Amplify Chronos: Public Discourse Timeline (Console Version) ---")

    # Fetch data (now exclusively uses hardcoded data due to 403 error)
    full_df, overall_ai_brief = fetch_and_process_data_for_console(DEFAULT_QUERY_TEXT, DEFAULT_RESULT_SIZE)

    if full_df.empty:
        print(f"\nNo articles available to display: {overall_ai_brief}")
    else:
        # Display overall AI Brief
        print(f"\nOverall Brief for '{DEFAULT_QUERY_TEXT}':")
        print(overall_ai_brief)
        print("-" * 50)

        # Get user input for filtering
        print("\nEnter filter parameters:")
        while True:
            try:
                num_units = int(input("Go back by (number): "))
                if num_units <= 0:
                    raise ValueError
                break
            except ValueError:
                print("Invalid input. Please enter a positive whole number.")

        while True:
            unit_type = input("Time Unit (days, months, years): ").lower()
            if unit_type in ['days', 'months', 'years']:
                break
            else:
                print("Invalid unit type. Please choose 'days', 'months', or 'years'.")

        # Apply filtering
        filtered_df = filter_df_by_time(full_df, num_units, unit_type)

        print(f"\n--- Chronological Timeline for '{DEFAULT_QUERY_TEXT}' (Last {num_units} {unit_type}) ---")
        print(f"Total articles in this filtered period: {len(filtered_df)}")
        print(f"Filtering from current time: {datetime.now(pytz.timezone('Europe/London')).strftime('%Y-%m-%d %H:%M:%S %Z%z')}")
        print("-" * 50)

        if not filtered_df.empty:
            for index, row in filtered_df.iterrows():
                formatted_timestamp = row['timestamp'].strftime('%Y-%m-%d %H:%M %Z')
                print(f"[{formatted_timestamp}]")
                print(f"Title: {row['title']}")
                print(f"Summary: {row['summary']}")
                print(f"URL: {row['url']}")
                if row['highlights']:
                    print("  Highlights:")
                    for highlight in row['highlights']:
                        print(f"  - {highlight}")
                print("-" * 50)
        else:
            print(f"No articles found for '{DEFAULT_QUERY_TEXT}' in the last {num_units} {unit_type}.")

--- Amplify Chronos: Public Discourse Timeline (Console Version) ---
Live API access is currently forbidden (403 error). Falling back to static data for demo purposes.

Overall Brief for 'President of US (Using static data)':
This is a static, simplified summary due to live API access issues. It demonstrates interactions between key public figures.
--------------------------------------------------

Enter filter parameters:
Go back by (number): 7
Time Unit (days, months, years): days

--- Chronological Timeline for 'President of US (Using static data)' (Last 7 days) ---
Total articles in this filtered period: 3
Filtering from current time: 2025-06-10 14:23:46 BST+0100
--------------------------------------------------
[2025-06-05 15:10 BST]
Title: Musk Says Trump Won't Be President Without His Help
Summary: Elon Musk claimed Donald Trump would not achieve presidency without his support, sparking public debate.
URL: https://example.com/musk-trump-help
  Highlights:
  - Musk: Trump will 

In [28]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import pytz # For timezone handling

# --- Configuration ---
API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "OPQ38746G38B7RB46GBER" # Our team's API Key
DEFAULT_QUERY_TEXT = "US president Trump" # <--- Using the query that reliably returned data
DEFAULT_RESULT_SIZE = 20 # <--- Using a safe result size for stability

# --- Function to fetch and process data (LIVE API CALL) ---
def fetch_and_process_data_for_console(query_text, result_size):
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_KEY
    }
    payload = {
      "query_text": query_text,
      "result_size": result_size,
      "include_highlights": True,
    }

    try:
        print(f"Attempting LIVE API call for query: '{query_text}' with result_size: {result_size}...")
        response = requests.post(API_URL, headers=headers, data=json.dumps(payload), timeout=30)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        json_response = response.json()

        if 'results' in json_response:
            df_live = pd.json_normalize(json_response['results'])

            # --- IMPORTANT: Convert timestamp to datetime AND handle NaT here ---
            df_live['timestamp'] = pd.to_datetime(df_live['timestamp'], errors='coerce')
            df_live.dropna(subset=['timestamp'], inplace=True)
            # --- End of IMPORTANT timestamp handling ---

            print(f"Live API call successful. Retrieved {len(df_live)} articles.")
            overall_ai_brief_live = json_response.get('ai_answer', {}).get('text', "AI summary not available or encountered an API error.")
            if "Unable to generate AI answer due to API error" in overall_ai_brief_live:
                overall_ai_brief_live = "AI summary encountered an error from the API. We will focus on the raw article data."
            return df_live, overall_ai_brief_live
        else:
            print(f"Live API call failed: 'results' key not found in response. Response: {json_response}")
            return pd.DataFrame(), "API call failed or returned unexpected structure. No articles retrieved."
    except requests.exceptions.Timeout:
        print("API request timed out. Could not retrieve live data.")
        return pd.DataFrame(), "API request timed out. No articles retrieved."
    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}. Could not retrieve live data.")
        return pd.DataFrame(), f"Error retrieving data: {e}. No articles retrieved."
    except json.JSONDecodeError:
        print("Could not decode JSON from API response. Response was not valid JSON.")
        return pd.DataFrame(), "Error: API response was not valid JSON. No articles retrieved."


# --- Function to filter DataFrame by time (same as before) ---
def filter_df_by_time(dataframe, num_units, unit_type):
    london_tz = pytz.timezone('Europe/London')
    current_date = datetime.now(london_tz)

    if unit_type == 'days':
        start_date = current_date - timedelta(days=num_units)
    elif unit_type == 'months':
        start_date = current_date - timedelta(days=num_units * 30) # Approximate
    elif unit_type == 'years':
        start_date = current_date - timedelta(days=num_units * 365) # Approximate
    else:
        print("Invalid unit_type. Please choose 'days', 'months', or 'years'.")
        return dataframe

    # Ensure DataFrame timestamps are timezone-aware and converted to London timezone
    if dataframe['timestamp'].dt.tz is None:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_localize('UTC').dt.tz_convert(london_tz)
    else:
        dataframe['timestamp'] = dataframe['timestamp'].dt.tz_convert(london_tz)

    filtered_df = dataframe[dataframe['timestamp'] >= start_date].copy()
    return filtered_df.sort_values(by='timestamp', ascending=True)

# --- Main execution block for console output ---
if __name__ == "__main__":
    print("--- Amplify Chronos: Public Discourse Timeline (Console Version) ---")

    # Fetch data directly from the live API
    full_df, overall_ai_brief = fetch_and_process_data_for_console(DEFAULT_QUERY_TEXT, DEFAULT_RESULT_SIZE)

    if full_df.empty:
        print(f"\nCould not retrieve any articles for '{DEFAULT_QUERY_TEXT}'. {overall_ai_brief}")
    else:
        # Display overall AI Brief
        print(f"\nOverall Brief for '{DEFAULT_QUERY_TEXT}':")
        print(overall_ai_brief)
        print("-" * 50)

        # Get user input for filtering
        print("\nEnter filter parameters:")
        while True:
            try:
                num_units = int(input("Go back by (number): "))
                if num_units <= 0:
                    raise ValueError
                break
            except ValueError:
                print("Invalid input. Please enter a positive whole number.")

        while True:
            unit_type = input("Time Unit (days, months, years): ").lower()
            if unit_type in ['days', 'months', 'years']:
                break
            else:
                print("Invalid unit type. Please choose 'days', 'months', or 'years'.")

        # Apply filtering
        filtered_df = filter_df_by_time(full_df, num_units, unit_type)

        print(f"\n--- Chronological Timeline for '{DEFAULT_QUERY_TEXT}' (Last {num_units} {unit_type}) ---")
        print(f"Total articles in this filtered period: {len(filtered_df)}")
        print(f"Filtering from current time: {datetime.now(pytz.timezone('Europe/London')).strftime('%Y-%m-%d %H:%M:%S %Z%z')}")
        print("-" * 50)

        if not filtered_df.empty:
            for index, row in filtered_df.iterrows():
                formatted_timestamp = row['timestamp'].strftime('%Y-%m-%d %H:%M %Z')
                print(f"[{formatted_timestamp}]")
                print(f"Title: {row['title']}")
                print(f"Summary: {row['summary']}")
                print(f"URL: {row['url']}")
                if row['highlights']:
                    print("  Highlights:")
                    for highlight in row['highlights']:
                        print(f"  - {highlight}")
                print("-" * 50)
        else:
            print(f"No articles found for '{DEFAULT_QUERY_TEXT}' in the last {num_units} {unit_type}.")

--- Amplify Chronos: Public Discourse Timeline (Console Version) ---
Attempting LIVE API call for query: 'US president Trump' with result_size: 20...
Live API call successful. Retrieved 15 articles.

Overall Brief for 'US president Trump':
AI summary not available or encountered an API error.
--------------------------------------------------

Enter filter parameters:
Go back by (number): 7
Time Unit (days, months, years): days

--- Chronological Timeline for 'US president Trump' (Last 7 days) ---
Total articles in this filtered period: 14
Filtering from current time: 2025-06-10 14:54:27 BST+0100
--------------------------------------------------
[2025-06-03 18:52 BST]
Title: Michigan Gov. Gretchen Whitmer makes thoughts about kidnapping plot known to President Donald Trump
Summary: Manage your account
Michigan Gov. Gretchen Whitmer said she talked to President Donald Trump in the last 24 hours about the two men convicted of plotting to kidnap her (AP video: Mike Householder)
Advertise